# CSV Prep
Process the 30 H&E WSI from Martinez.
Previously, the CellProfiler Process100 pipeline ran on WSI patches in batches.
The batches were designed to meet processing time requirements,
with several patients per batch as time allowed.
Here, we reorganize the data by patient.

In [9]:
import numpy as np
import pandas as pd
import csv

In [10]:
BASE_PATH='/home/jrm/Martinez/CellProfilerRuns/CP_20220705/'
INPUT_DIRS=[
'HEcenterpatch.train.Yneg.A3-A5/',
'HEcenterpatch.train.Yneg.B13/',
'HEcenterpatch.train.Yneg.C11-D3/',
'HEcenterpatch.train.Yneg.E5/',
'HEcenterpatch.train.Yneg.F3-F15/',
'HEcenterpatch.train.Yneg.G15-H15/',
'HEcenterpatch.train.Ypos.B7-D5/',
'HEcenterpatch.train.Ypos.E7-F9/',
'HEcenterpatch.train.Ypos.G3-I13/',
'HEcenterpatch.valid.Yneg/',
'HEcenterpatch.valid.Ypos/']
FILENAMES=[
'Process100_Image.csv',
'Process100_Cells.csv',
'Process100_ExpandCells.csv',
'Process100_Experiment.csv',
'Process100_MergeRBC.csv',
'Process100_Nucleus.csv',
'Process100_RBC.csv',
'Process100_ShrinkRBC.csv',
'Process100_Tissue.csv']

In [11]:
filename = BASE_PATH+INPUT_DIRS[0]+FILENAMES[0]
filename

'/home/jrm/Martinez/CellProfilerRuns/CP_20220705/HEcenterpatch.train.Yneg.A3-A5/Process100_Image.csv'

In [12]:
with open(filename) as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        print(row)
        break

{'AreaOccupied_AreaOccupied_ExpandCells': '50176.0', 'AreaOccupied_AreaOccupied_MergeRBC': '0.0', 'AreaOccupied_AreaOccupied_Nucleus': '2202.0', 'AreaOccupied_AreaOccupied_Tissue': '40323.0', 'AreaOccupied_Perimeter_ExpandCells': '2419.0', 'AreaOccupied_Perimeter_MergeRBC': '0.0', 'AreaOccupied_Perimeter_Nucleus': '414.0', 'AreaOccupied_Perimeter_Tissue': '1174.0', 'AreaOccupied_TotalArea_ExpandCells': '50176.0', 'AreaOccupied_TotalArea_MergeRBC': '50176.0', 'AreaOccupied_TotalArea_Nucleus': '50176.0', 'AreaOccupied_TotalArea_Tissue': '50176.0', 'Channel_Tumor': '-2', 'Count_Cells': '5.0', 'Count_ExpandCells': '5.0', 'Count_MergeRBC': '0.0', 'Count_Nucleus': '5.0', 'Count_RBC': '0.0', 'Count_ShrinkRBC': '0.0', 'Count_Tissue': '1.0', 'ExecutionTime_01Images': '0.0', 'ExecutionTime_02Metadata': '0.0', 'ExecutionTime_03NamesAndTypes': '0.734375', 'ExecutionTime_04Groups': '0.0', 'ExecutionTime_05UnmixColors': '0.0625', 'ExecutionTime_06Smooth': '0.0', 'ExecutionTime_07Smooth': '0.0', 'Exe

In [18]:
TUMOR_COL='FileName_Tumor'
df = pd.read_csv(filename)
patients = df[TUMOR_COL].astype(str).str[:3].unique()
patients

array(['A3_', 'A5_'], dtype=object)